# __Spherical harmonic transform__ 
---

[![colab image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astro-informatics/s2fft/blob/main/notebooks/spherical_harmonic_transform.ipynb)

In [1]:
# Install s2fft
!pip install s2fft &> /dev/null

This tutorial demonstrates how to use `S2FFT` to compute spherical harmonic transforms. Specifically, we will adopt the sampling scheme of [McEwen & Wiaux (2012)](https://arxiv.org/abs/1110.6298). 

First let's load an input signal that is sampled on the sphere with this sampling scheme.

In [2]:
import jax
jax.config.update("jax_enable_x64", True)

import numpy as np
import s2fft 

L = 256
sampling = "mw"
flm = np.random.randn(L, 2*L-1) + 1j*np.random.randn(L, 2*L-1)
f = s2fft.inverse_jax(flm, L)

### Computing the forward spherical harmonic transform

---

Let's now run the JAX function to compute the spherical harmonic transform of this map.

In [3]:
flm = s2fft.forward_jax(f, L)

If you are planning on applying this transform many times (e.g. during training of a model) we recommend precomputing and storing some small arrays that are used every time. To do this simply compute these and pass as a static argument.

In [4]:
precomps = s2fft.generate_precomputes_jax(L, forward=True)
flm_pre = s2fft.forward_jax(f, L, precomps=precomps)

### Computing the inverse spherical harmonic transform

---

Let's run the JAX function to cmpute the inverse spherical harmonic transform to get back to the input map.

In [5]:
f_recov = s2fft.inverse_jax(flm, L)

Again, if you are planning on applying this transform many times we recommend precomputing and storing some small arrays that are used every time. To do this simply compute these and pass as a static argument.

In [6]:
precomps = s2fft.generate_precomputes_jax(L, forward=False)
f_recov_pre = s2fft.inverse_jax(flm_pre, L, precomps=precomps)

### Computing the error

---

Let's check the associated error, which should be close to machine precision for the sampling scheme used.

In [8]:
print(f"Mean absolute error = {np.nanmean(np.abs(f_recov - f))}")

Mean absolute error = 8.440183049951758e-11


In [9]:
print(f"Mean absolute error using precomputes = {np.nanmean(np.abs(f_recov_pre - f))}")

Mean absolute error using precomputes = 8.440183049951758e-11
